# Covid nas cidades
Esse script prepara os dados brutos para a visualização de dados.

## Importação de pacotes

In [81]:
import json, unidecode
import math
import numpy as np
import pandas as pd

## Acessa os dados

### Fontes: 

- Registro de casos: [Brasil.IO](https://github.com/wcota/covid19br/issues/17#issuecomment-609113071)

- Divisão do território: [IBGE](https://servicodados.ibge.gov.br/api/docs/localidades#api-Municipios-municipiosGet), compilada em CSV por Cristiano Martins, voluntário do Brasil.IO

In [82]:
df = pd.read_csv("../cases/caso-full.csv", dtype={"city_ibge_code":str })

In [83]:
geo = pd.read_csv("../ibge/municipios.csv", dtype=
                  {
                   "cod_municipio": str, 
                   "cod_meso": str, 
                   "cod_micro": str, 
                   "cod_uf": str
                  })

In [84]:
geo.head()

,cod_municipio,nome_município,cod_uf,nome_uf,cod_meso,nome_meso,cod_micro,nome_micro,latitude,longitude
0,5200050,Abadia de Goiás,52,Goiás,03,Centro Goiano,010,Goiânia,-167573.0,-494412.0
1,3100104,Abadia dos Dourados,31,Minas Gerais,05,Triângulo Mineiro/Alto Paranaíba,019,Patrocínio,-184831.0,-473916.0
2,5200100,Abadiânia,52,Goiás,04,Leste Goiano,012,Entorno de Brasília,-16197.0,-487057.0
3,3100203,Abaeté,31,Minas Gerais,06,Central Mineira,024,Três Marias,-191551.0,-454444.0
4,1500107,Abaetetuba,15,Pará,04,Nordeste Paraense,011,Cametá,-172183.0,-488788.0


## Tratamento inicial

Mantém apenas os pontos de dados com mais de **n** casos - ou seja, as cidades só vão aparecer no gráfico quando tiverem mais de **n** registros.

In [6]:
n = 1

In [7]:
df = df [ df.last_available_confirmed >= n ]

Extrai o código das UFs, microrregiões e mesorregiões e UFs com um merge.

In [8]:
df = df.merge(geo, left_on="city_ibge_code", right_on="cod_municipio")

Remove colunas desnecessárias.

In [9]:
df = df.drop(['cod_municipio', 'nome_município', 'nome_uf', 'latitude', 'longitude', 
              'last_available_death_rate', 'last_available_confirmed_per_100k_inhabitants'], axis=1)

Cria o código único de microrregião e mesorregião, combinando os indetificadores.

In [10]:
df.cod_meso = df.cod_uf + df.cod_meso
df.cod_micro = df.cod_meso + df.cod_micro

Agrupa os dados por microrregião e data. 

É importante notar que, quando ao menos um campo booleano for verdadeiro, a soma vai retornar um valor maior que zero. Isso vai criar redundância no campo 'is_last', que será corrigida futuramente. Entretanto, vai ser útil pra saber as datas em que não houve alteração em nenhum dos boletins das cidades da microrregião. Nesses casos, e somente nestes, is_fake será True.

In [11]:
df = df.groupby(["cod_micro", "cod_meso", "cod_uf", "state", "nome_micro", "nome_meso", "date"]).sum().reset_index()

Converte os campos novamente para booleano.

In [12]:
def back_to_bool(row):
    
    if row.is_last < 1:
        is_last = False

    else:
        is_last = True
        
    if row.is_repeated < 1:
        is_fake = False
        
    else:
        is_fake = True
            
    return pd.Series({"is_last": is_last, "is_fake": is_fake})

In [13]:
df[["is_last", "is_fake"]] = df.apply(back_to_bool, axis=1)

Como a referência de data são as cidades e não as microrregiões, acabamos com mais de uma data 'is_last'. Vamos manter apenas a última delas.

In [14]:
# Reordena por cidade e data
df = df.sort_values(by=["cod_micro", "date"])

In [15]:
def fix_is_last(df):
    
    '''
    TO DO: otimizar função para melhorar tempo de execução.
    Uma saída pode ser selecionar apenas as linhas marcadas
    como is_last, mas há inconsistências na base (como uma data
    posterior a última is_last marcada como False) que impedem
    essa saída. Por enquanto, consigo viver com 30s de execução.
    '''
    
    # Para cada microrregião...
    for micro in df.cod_micro.unique():
        
        # Selecione as entradas marcadas como is_last
        temp = df[ (df.cod_micro == micro)]
                
        # Pegue a data mais recente
        last_date = temp.date.max()
                
        # Itere pelas linhas
        for index, row in temp.iterrows():
            
            # Verifique se é de fato a última data. Se for, retorne True. Se não, retorne False
            temp.loc[index] = True if row.date == last_date else False
        
        # Substitua no df prinicpal usando o índice
        df.loc[temp.index, 'is_last'] = temp.is_last 
    
    # Retorne o df completo
    return df

In [16]:
%%time
df = fix_is_last(df)

/home/rodrigo/anaconda3/envs/coronavirus/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


CPU times: user 14.9 s, sys: 4.28 ms, total: 14.9 s
Wall time: 14.8 s


Recalcula as taxas de casos pro habitante e mortalidade

In [17]:
df["confirmed_per_100k"] = round(df.last_available_confirmed / df.estimated_population_2019 * 100000, 3)

In [18]:
df["cfr"] = df.last_available_deaths / df.last_available_confirmed

Calcula novos casos e mortes por dia em uma rolling average de 7 dias para cada cidade

In [19]:
%%time
for cod_micro in df.cod_micro.unique():
     
        temp = df[df.cod_micro == cod_micro]
        
        temp["new_confirmed_rolling"] = temp.new_confirmed.rolling(5).mean()
        temp["new_deaths_rolling"] = temp.new_deaths.rolling(5).mean()
                  
        df.loc[temp.index, "new_confirmed_rolling"] = temp.new_confirmed_rolling
        df.loc[temp.index, "new_deaths_rolling"] = temp.new_deaths_rolling



<timed exec>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


CPU times: user 1.06 s, sys: 15 µs, total: 1.06 s
Wall time: 1.05 s


Calcula crescimento percentual de casos e mortes dia a dia e depois faz a rolling average

In [20]:
df["confirmed_growth_rate"] = df.new_confirmed / (df.last_available_confirmed - df.new_confirmed)
df["deaths_growth_rate"] = df.new_deaths / (df.last_available_deaths - df.new_deaths)

Nessas taxas, o primeiro caso registrado vira um crescimento infinito (divisão por zero). Nesses casos, vamos representar o crescimento como 0% (momento de saída)

In [21]:
def no_inf(row):
    
    if np.isinf(row.confirmed_growth_rate):
        confirmed_growth_rate = 0
        
    else:
        confirmed_growth_rate = row.confirmed_growth_rate
        
    if np.isinf(row.deaths_growth_rate):
        deaths_growth_rate = 0
        
    else:
        deaths_growth_rate = row.deaths_growth_rate
        
    return pd.Series({
        "confirmed_growth_rate": confirmed_growth_rate,
        "deaths_growth_rate": deaths_growth_rate
    })

In [22]:
%%time
df[["confirmed_growth_rate", "deaths_growth_rate"]] = df.apply(no_inf, axis=1)

CPU times: user 1.95 s, sys: 4 ms, total: 1.95 s
Wall time: 1.95 s


Calcula uma média rolante

In [23]:
%%time
for cod_micro in df.cod_micro.unique():
     
        temp = df[df.cod_micro == cod_micro]
        
        temp["confirmed_growth_rate_rolling"] = temp.confirmed_growth_rate.rolling(5).mean()
        temp["deaths_growth_rate_rolling"] = temp.deaths_growth_rate.rolling(5).mean()
                  
        df.loc[temp.index, "confirmed_growth_rate_rolling"] = temp.confirmed_growth_rate_rolling
        df.loc[temp.index, "deaths_growth_rate_rolling"] = temp.deaths_growth_rate_rolling



<timed exec>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


CPU times: user 1.05 s, sys: 29 µs, total: 1.05 s
Wall time: 1.05 s


Arredonda casas decimais

In [24]:
df = df.round(3)

In [25]:
to_inspect = [
    "date",
    "last_available_confirmed",
    "new_confirmed",
    "new_confirmed_rolling",
    "confirmed_growth_rate",
    "confirmed_growth_rate_rolling", 
    "last_available_deaths",
    "new_deaths",
    "new_deaths_rolling", 
    "deaths_growth_rate", 
    "deaths_growth_rate_rolling",
    

]

Corrige manualmente erro pontual na região de Fortaleza

In [26]:
df.loc[563, "new_deaths"] = 0
df.loc[563, "last_available_deaths"] = df.loc[562, "last_available_deaths"]

Para cada região, calcula o total de dias passados desde o primeiro contágio

In [27]:
for cod_micro in df.cod_micro.unique():

    temp = df[df.cod_micro == cod_micro]
    
    temp.days_since_outbreak = 0
    
    count = 0
    for index, row in temp.iterrows():
        
        temp.loc[index, "days_since_outbreak"] = count
        count += 1
        
    df.loc[temp.index, "days_since_outbreak"] = temp.days_since_outbreak

/home/rodrigo/anaconda3/envs/coronavirus/lib/python3.8/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/rodrigo/anaconda3/envs/coronavirus/lib/python3.8/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


Recorta as microrregiões mais populosas

In [28]:
over_500k = df[df.estimated_population_2019 > 500000]

Salva para CSV

In [29]:
df.to_csv("../processed/microregions.csv", index = False)

In [30]:
over_500k.to_csv("../processed/microregions_over_500k.csv", index = False)

## Prepara arquivo JSON com chave de busca

In [31]:
geo.cod_meso = geo.cod_uf + geo.cod_meso
geo.cod_micro = geo.cod_meso + geo.cod_micro

In [32]:
geo.head()

,cod_municipio,nome_município,cod_uf,nome_uf,cod_meso,nome_meso,cod_micro,nome_micro,latitude,longitude
0,5200050,Abadia de Goiás,52,Goiás,5203,Centro Goiano,5203010,Goiânia,-167573.0,-494412.0
1,3100104,Abadia dos Dourados,31,Minas Gerais,3105,Triângulo Mineiro/Alto Paranaíba,3105019,Patrocínio,-184831.0,-473916.0
2,5200100,Abadiânia,52,Goiás,5204,Leste Goiano,5204012,Entorno de Brasília,-16197.0,-487057.0
3,3100203,Abaeté,31,Minas Gerais,3106,Central Mineira,3106024,Três Marias,-191551.0,-454444.0
4,1500107,Abaetetuba,15,Pará,1504,Nordeste Paraense,1504011,Cametá,-172183.0,-488788.0


In [33]:
data = [ ]

for region_id in df.cod_micro.unique():
    
    cities_within = geo [ geo.cod_micro == region_id].nome_município.unique()
    cities_within = sorted(cities_within)
    cities_within = ", ".join(cities_within)
                            
    # For better searching on the JS
    cities_within_unidecoded = unidecode.unidecode(cities_within)

    region_name = geo [ geo.cod_micro == region_id ].nome_micro.unique()[0]
    state = df [ df.cod_micro == region_id ].state.unique()[0]
    
    obj = {
        "regionName": f"{region_name} ({state})",
        "citiesWithin": cities_within,
        "citiesWithinUnidecoded": cities_within_unidecoded,
        "regionId": region_id
    }
    
    data.append(obj)

In [34]:
with open("../processed/search-keys.json",
          "w+", 
          encoding="utf-8") as file:
    json.dump(data, file, ensure_ascii=False, indent=4)